# Chapter 7 Sentiment Analysis
감정분석

In [1]:
# 문장 시퀀스 뒤에 감정을 결정하는 과정으로 정의한다
# Speaker 혹은 Text사고를 표현하는 사람의 감정을 판단하는데 사용

In [2]:
# 1 감정분석 소개
# 2 NER을 사용한 감정분석
# 3 기계학습을 활용한 감정분석
# 4 NER 시스템의 평가

## 1 Introduction
감정분석 소개

In [3]:
# target : 이진분류(긍정, 부정), 멀티분류 (긍정, 부정, 중립)
# 감정과 토픽 마이닝을 결합한 '토픽-감정분석'을 시행한다

In [4]:
# 감정분석 : lexicon (어휘목록) 을 사용해서 수행할 수 있다
# 1 labMT (10,000단어 분석)
# 2 Warringer (13,915단어 분석)
# 3 OpinionFinder's Subjectivity Lexic (8221단어 분석)
# 4 ANEW  (1034단어 분석)    : Affective Norms for English Words
# 5 AFINN (2477단어 분석)    : Finn Arup Nielson 에 의한 분류
# 6 Balance Affective (277 단어) : 1(긍정), 2(부정), 3(불안정), 4(중립)
# 7 BAWL  (2200단어 분석)    : Berlin Affective Word List Reloaded
# 8 BFAN  (210단어로 구성)    : Bilingual Finnish Affective Norms
# 9 CDGE  : Compass DeRose Guide to Emotion Words
# 10 DAL  : Dictionary of Affect in Language
# 11 WDAL : Whissell's Dictionary of Affect in Language
# 등등...

## 2 영어 리뷰의 감정분석
sentiment analysis for movie review

In [5]:
from nltk.corpus import movie_reviews
movie_reviews.categories()

['neg', 'pos']

In [6]:
movie_reviews.fileids(movie_reviews.categories()[0])[:5]

['neg/cv000_29416.txt',
 'neg/cv001_19502.txt',
 'neg/cv002_17424.txt',
 'neg/cv003_12683.txt',
 'neg/cv004_12641.txt']

In [7]:
docs = [(list(movie_reviews.words(fid)), cat) 
        for cat in movie_reviews.categories()   # ['neg', 'pos']
        for fid in movie_reviews.fileids(cat)]  # 'neg/cv000_29416.txt',....

# 2000개 문서, 긍/부정 분류, 개별 문서의 token 목록으로 정리
print(len(docs), len(docs[0]), len(docs[0][0]))
print(len(docs), len(docs[13]), len(docs[13][0]))

2000 2 879
2000 2 1144


In [8]:
# 긍/부정 분류된 textdml token을 뒤섞어서 1개로 생성
import nltk, random

random.shuffle(docs)
all_tokens = nltk.FreqDist(x.lower() for x in movie_reviews.words())
print('영화리뷰 token의 총 갯수 :', len(all_tokens.keys()))

token_features = list(all_tokens.keys())[:2000]
token_features[::200] 

영화리뷰 token의 총 갯수 : 39768


['plot',
 'arrow',
 'mir',
 'indication',
 'seymour',
 'house',
 'digital',
 'terrible',
 'strives',
 'willing']

In [9]:
# nltk의 영화리뷰 테이터 묶음 'token_features'에 'docs'데이터가 포함여부를 판단
# Accuracy Boolean 사용자 함수

def doc_features(docs):
    doc_words = set(docs)  # 대상 문서의 token을 집합으로 추출
    features = {}
    for word in token_features:
        features['contains(%s)' % word] = (word in doc_words)
        return features

token_file = 'pos/cv957_8737.txt' 
print(token_file + 's Token :', len(movie_reviews.words(token_file)))
doc_features(movie_reviews.words( token_file ))

pos/cv957_8737.txts Token : 597


{'contains(plot)': True}

In [10]:
# docs : nltk 영화리뷰의 모든 token
feature_sets = [(doc_features(d), c) for (d,c) in docs]

# 2000개 뒤섞은 리뷰 100개씩 추출하여 train/ test 데이터를 생성
train_sets, test_sets = feature_sets[100:], feature_sets[:100]

# 나이브 베이즈 분류기로 훈련데이터 만들기
classifiers = nltk.NaiveBayesClassifier.train(train_sets)

# 나이브 베이즈 분류기 자료와, test_set의 정확도 판단  
print(nltk.classify.accuracy(classifiers, test_sets))
classifiers.show_most_informative_features(5) 

0.6
Most Informative Features
          contains(plot) = True              neg : pos    =      1.4 : 1.0
          contains(plot) = False             pos : neg    =      1.3 : 1.0


In [11]:
# 결과
# 정보 특징 (Most Informative Features) 이  문서내 존재여부를 체크한다
# (5) 는 없어도 결과는 위처럼 2개만 결과로 출력....
# plot데 대해서만 귀무가설, 대립가설 일치도를 판단
# 책과 다르다.... GitHub의 결과도 위와 동일.....


## 3 텍스트 전처리
nltk

In [12]:
# text --> 문장 --> token
import nltk
class Splitter(object):
    
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        sentences = self.nltk_splitter.tokenize(text)  # text --> 문장
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) # 문장 --> token
                               for sent in sentences]
        return tokenized_sentences

# 문장의 3개 token으로 묶어서 정리한다
# 단어(word), 표제어(lemme), 태그(tag)
class POSTagger(object):
    
    def __init__(self):
        pass
    
    def pos_tag(self, sentences):
        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        pos = [[(word, word, [postag]) 
                for (word, postag) in sentence] 
                for sentence in pos]
        return pos

In [13]:
text = """Why are you looking disappointed. 
We will go to restaurant for dinner."""

splitter = Splitter()
postagger = POSTagger()
splitted_sentences = splitter.split(text)
splitted_sentences

[['Why', 'are', 'you', 'looking', 'disappointed', '.'],
 ['We', 'will', 'go', 'to', 'restaurant', 'for', 'dinner', '.']]

In [14]:
pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
pos_tagged_sentences

[[('Why', 'Why', ['WRB']),
  ('are', 'are', ['VBP']),
  ('you', 'you', ['PRP']),
  ('looking', 'looking', ['VBG']),
  ('disappointed', 'disappointed', ['VBN']),
  ('.', '.', ['.'])],
 [('We', 'We', ['PRP']),
  ('will', 'will', ['MD']),
  ('go', 'go', ['VB']),
  ('to', 'to', ['TO']),
  ('restaurant', 'restaurant', ['VB']),
  ('for', 'for', ['IN']),
  ('dinner', 'dinner', ['NN']),
  ('.', '.', ['.'])]]

In [15]:
# https://github.com/sujithvm/nlp-modules/blob/master/sentiment%20analysis/sentiment_analyzer.py
# 폴더내 파일들의 text에 대해 긍정/ 부정 tag 작업 시행
# dictionary를 활용해서 tag를 생성

class DictionaryTagger(object):

    def __init__(self, dictionary_paths):
        files = [open(path, 'r') for path in dictionary_paths]      
        map(lambda x: x.close(), files)

        dictionaries = [yaml.load(dict_file) for dict_file in files]
        self.dictionary = {}
        self.max_key_size = 0
        
        for curr_dict in dictionaries:
            for key in curr_dict:
                if key in self.dictionary:  # dictionary에 포함된 목록인 경우           
                    self.dictionary[key].extend(curr_dict[key])                    
                else:                       # 새로운 dictionary 목록인 경우
                    self.dictionary[key] = curr_dict[key] # dict을 생성/ 최대값 추가                   
                    self.max_key_size = max(self.max_key_size, len(key))
                
    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0: 
            self.max_key_size = N
        i = 0
        while (i < N):
            j = min(i + self.max_key_size, N)
            tagged = False
            while (j > i):
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()

                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form

                if literal in self.dictionary:
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token: 
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1

            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence

In [16]:
# 긍/부정 정리된 dictionary 목록의 표현들을 Counting
def value_of(sentiment):
    if sentiment == 'positive': return 1
    if sentiment == 'negative': return -1
    return 0

def sentiment_score(review):
    return sum([sentence_score(sentence, None, 0.0) for sentence in review])

## 4 NER를 사용한 감정분석 -  176 p
개체명(고유명사) 인식 Named-entity recognition (NER) - 감정식별을 위한 stopword 구분절차

    token의 개체명을 별도의 기준으로 식별 후, 클래스로 분류하는 과정으로 
    히든마르코프, 최대엔트로피 마르코프, SVM, 의사결정나무 등을 활용한다
    개체명으로 인식되면, 감정분석에 기여하지 않으므로, 제외한 나머지들로 감정분석을 수행

## 5 기계학습을 사용한 감정분석
nltk.sentiment.sentiment_analyzer() 는 기계학습 기반의 감정분석 모듈이다

In [17]:
#from __future__ import print_function
from collections import defaultdict
from nltk.classify.util import apply_features, accuracy as eval_accuracy
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import (BigramAssocMeasures, precision as eval_precision, 
                          recall as eval_recall, f_measure as eval_f_measure)
from nltk.probability import FreqDist
from nltk.sentiment.util import save_file, timer

# 기계학습에 기반한 감정분석도구
class SentimentAnalyzer(object):
    def __init__(self, classifier=None):
        self.feat_extractors = defaultdict(list)
        self.classifier = classifier

/home/markbaum/Python/python36/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [18]:
# 텍스트에서 모든 (중복)단어를 반환
def all_words(self, documents, labeled=None):
    all_words = []
    if labeled is None:
        labeled = documents and isinstance(documents[0], tuple)
    if labeled == True:
        for words, sentiment in documents:
            all_words.extend(words)
    elif labeled == False:
        for words in documents:
            all_words.extend(words)
    return all_words

In [19]:
# 특징 추출 함수 feature extraction function
def apply_features(self, documents, labeled=None):
    return apply_features(self.extract_features, documents,labeled)

In [20]:
# 단어의 특징을 반환하는 코드
def unigram_word_feats(self, words, top_n=None, min_freq=0):
    unigram_feats_freqs = FreqDist(word for word in words)
    return [w    for   w, f   in   unigram_feats_freqs.most_common(top_n) 
                 if   unigram_feats_freqs[w]  >  min_freq ]

In [21]:
# bi-gram의 특징을 반환하는 코드
def bigram_collocation_feats(self, documents, top_n=None, min_freq=3, assoc_measure=BigramAssocMeasures.pmi):
    finder = BigramCollocationFinder.from_documents(documents)
    finder.apply_freq_filter(min_freq)
    return finder.nbest(assoc_measure, top_n)

In [22]:
# 사용 가능한 특징세트를 사용하여, 주어진 인스턴스를 분류
def classify(self, instance):
    instance_feats = self.apply_features([instance],labeled=False)
    return self.classifier.classify(instance_feats[0])

In [23]:
# 텍스트에서 특징 추출을 위해 사용
def add_feat_extractor(self, function, **kwargs):
    self.feat_extractors[function].append(kwargs)

def extract_features(self, document):
    all_features = {}
    for extractor in self.feat_extractors:
        for param_set in self.feat_extractors[extractor]:
            feats = extractor(document, **param_set)
        all_features.update(feats)
    return all_features

In [24]:
# 훈련데이터를 훈련시키는 함수
def train(self, trainer, training_set, save_classifier = None, **kwargs):
    print("Training classifier")
    self.classifier = trainer(training_set, **kwargs)
    if save_classifier:
        save_file(self.classifier, save_classifier)
    return self.classifier

In [25]:
# 테스트데이터를 사용한 분류기의 테스트 및 성능평가
def evaluate(self, test_set, classifier = None, accuracy = True, 
             f_measure = True, precision = True, recall = True, verbose = False):

    if classifier is None:  # 분류기에 아무것도 지정하지 않은 경우
        classifier = self.classifier  # __init__의 초깃값을 불러와서 작업을 시작
    print("Evaluating {0} results...".format(type(classifier).__name__))
    metrics_results = {}              # 출력 report dictionary를 생성

    if accuracy == True:   # 정확도 측정옵션에 True를 입력시
        accuracy_score = eval_accuracy(classifier, test_set) # 분류기 기준, 데이터를 test한다
        metrics_results['Accuracy'] = accuracy_score         # report dictionary에 기록 
        
    # 출처 : https://dongyeopblog.wordpress.com/2016/04/08/python-defaultdict-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0/
    gold_results = defaultdict(set)   # key값 지정없어도, default로 key를 자동으로 지정한다
    test_results = defaultdict(set)   
    labels = set()
    for i, (feats, label) in enumerate(test_set): # test 상세내용을 기록한다
        labels.add(label)
        gold_results[label].add(i)
        observed = classifier.classify(feats)
        test_results[observed].add(i)

    for label in labels:  # test 결과 수집된 labels 에 따라, 평가함수로 계산을 한다
        if precision == True:  # 정확도 측정
            precision_score = eval_precision(gold_results[label], test_results[label])
            metrics_results['Precision [{0}]'.format(label)] = precision_score
        if recall == True:     # recall 측정
            recall_score = eval_recall(gold_results[label], test_results[label])
            metrics_results['Recall [{0}]'.format(label)] = recall_score
        if f_measure == True:  # f-measure 측정
            f_measure_score = eval_f_measure(gold_results[label], test_results[label])
            metrics_results['F-measure [{0}]'.format(label)] = f_measure_score
        if verbose == True:    # Data를 정렬
            for result in sorted(metrics_results):
                print('{0}: {1}'.format(result, metrics_results[result]))
    return metrics_results

## 6 기계학습을 사용한 감정분석
Twitter text 데이터에 대한 통계, 자동화, 기계학습 분류기

출처 : http://www.nltk.org/book/ch06.html

data : https://github.com/ravikiranj/twitter-sentiment-analyzer

<img src = "http://www.nltk.org/images/supervised-classification.png" align='left' width = '500'>

In [26]:
stopWords = []
import re, csv
# 중복되는 문자를 단일로 처리
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [27]:
# 불용어 목록을 파일에서 읽어온다
def getStopWordList(stopWordListFileName):
    stopWords = []   # 불용어 목록 파일의 text를 '공백'을 기준으로 token생성 뒤, list로 출력
    stopWords.append('AT_USER')
    stopWords.append('URL')
    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

In [28]:
# 여러번 반복된 단어들의 전처리
def getFeatureVector(tweet):
    featureVector = []
    words = tweet.split()       # 공백을 기분으로 token 단어를 생성
    for w in words:             
        w = replaceTwoOrMore(w) # 2번 이상 반복된 단어를 단일로 전처리 (사용자함수)
        w = w.strip('\'"?,.')   # 문장부호를 제거
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w) # 알파벳 여부 확인
    # 해당 단어가 불용어에 해당하면, 무시한다
    if(w in stopWords or val is None):
        pass # continue 는 '반복문'에서 가능
    else:
        featureVector.append(w.lower())
    return featureVector 

In [29]:
# https://gist.github.com/ravikiranj/2639031
# 트위터 데이터 전처리 함수
def processTweet(tweet):
    tweet = tweet.lower()  # 데이터를 소문자로 바꾼다
    # www.* 또는 https?://* 를 URL 로 변환
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet = re.sub('@[^\s]+','AT_USER',tweet)   # @username 을 AT_USER로 변환
    tweet = re.sub('[\s]+', ' ', tweet)         # 불필요한 여백제거   
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)  # 해시태그 '#' 를 제거
    tweet = tweet.strip('\'"')                  # trim
    return tweet

In [30]:
# 트위터 데이터가 .txt인 경우
# Tweets 데이터는 1줄씩 모든 프로세스를 진행한다
fp = open('data/sampleTweets.txt', 'r')           # Data text 파일
line = fp.readline()
st = open('data/stopwords.txt', 'r') # Stop word text 파일 
stopWords = getStopWordList('data/stopwords.txt')
while line:  # 트위터 데이터 1줄씩 처리
    processedTweet = processTweet(line)
    featureVector = getFeatureVector(processedTweet)
    print(featureVector)
    line = fp.readline()
fp.close() 

[]
[]
[]
['twitter']
[]
['makeitcount']
['sigh']
['hurts']
['hurts']


In [31]:
# 트위터 데이터가 .CSV 인 경우
# Tweets are read one by one and then processed.
inpTweets = csv.reader(open('./data/sampleTweets.csv', 'r'), delimiter=',', quotechar='|')
tweets = []

for row in inpTweets:
    sentiment = row[0]
    tweet = row[1]
    processedTweet = processTweet(tweet)
    featureVector = getFeatureVector(processedTweet) #, stopWords)
    tweets.append((featureVector, sentiment));

tweets

[([], 'positive'),
 ([], 'positive'),
 ([], 'positive'),
 (['twitter'], 'neutral'),
 ([], 'neutral'),
 (['makeitcount'], 'neutral'),
 (['sigh'], 'negative'),
 (['hurts'], 'negative'),
 (['hurts'], 'negative')]

In [32]:
# 특징 추출하는 메서드
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [33]:
# 나이브베이즈 분류기로 감정분석
NaiveBClassifier = nltk.NaiveBayesClassifier.train(train_sets)
# Testing the classifiertestTweet = 'I liked this book on Sentiment Analysis a lot.'
# processedTestTweet = processTweet(test_sets)
# NaiveBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))

In [34]:
testTweet = 'I am so badly hurt'
processedTestTweet = processTweet(testTweet)
# NaiveBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))

## 7 NER 시스템의 평가
Evaluation of the NER system